In [1]:
#get stats
from dask.diagnostics import ProgressBar
import numpy as np
import pickle as pkl
import gzip
from numba import jit
from multiprocessing import Process


# List of input files
# Open the file in read mode
with open('coverage_file.list', 'r') as file:
# Create an empty list to store the lines
    mapqfiles = []
    # Iterate over the lines of the file
    for line in file:
        # Remove the newline character at the end of the line
        line = line.strip()

        # Append the line to the list
        mapqfiles.append(line)

In [10]:
# Function to process a single file and extract a specific column by name
def process_file(input_file, col_index):
    with gzip.open(input_file, 'rt') as f:
            column = np.loadtxt(input_file, delimiter='\t', skiprows=1, usecols=col_index)  # Load data from the file, skipping the header
            #olumn = data[:, col_index]  # Select the specified column
            return column  # Remove the first row of the column

In [14]:
def infer_coverage_stats():
    if __name__ == "__main__":
        #get modal coverages
        with open('data/modes.pkl', 'rb') as f:
            modes = pkl.load(f)

        def return_ind_filter(array, mode):
            zerocov = array == 0 
            lowcov = array < (mode/2)
            hicov = array > (mode*2)
            return(zerocov, lowcov, hicov)

        #for all inds
        #make arrays containing all of the coverage stats
        #then sum acdross them to get proportion of individuals pass/fail each condition

        def return_summed_arrays():

            zero_cov_arrays = []
            low_cov_arrays = []
            hi_cov_arrays = []

            for mode, file in zip(modes, mapqfiles):

                arr = process_file(file, col_index = 2)

                print(f"Calculating coverage stats for {file}, with a modal cov of {mode}")

                zerocov, lowcov, hicov = return_ind_filter(arr, mode)

                zero_cov_arrays.append(zerocov)
                low_cov_arrays.append(lowcov)
                hi_cov_arrays.append(hicov)

            zero_cov_arrays = np.stack(zero_cov_arrays)
            low_cov_arrays = np.stack(low_cov_arrays)
            hi_cov_arrays = np.stack(hi_cov_arrays)

            zero_cov_bool = np.sum(zero_cov_arrays, axis=0) / len(mapqfiles)
            low_cov_bool = np.sum(low_cov_arrays, axis=0) / len(mapqfiles)
            hi_cov_bool = np.sum(hi_cov_arrays, axis=0) / len(mapqfiles)

            return(zero_cov_bool, low_cov_bool, hi_cov_bool)


        z,l,h = return_summed_arrays()

        # open a file, where you ant to store the data
        with open('data/coverage_summary_stats.pkl', 'wb') as f:
            pkl.dump([z,l,h], f)

IndentationError: expected an indented block after 'if' statement on line 2 (397862836.py, line 4)

In [ ]:
def infer_mapq_stats():
    if __name__ == "__main__":

        #for all inds
        #make arrays containing all of the coverage stats
        #then sum acdross them to get proportion of individuals pass/fail each condition

        def return_summed_arrays():

            mapq_arrays = []

            for file in mapqfiles:

                arr = process_file(file, col_index = 7)

                mapq = arr < 30

                mapq_arrays.append(mapq)

            mapq_arrays = np.stack(mapq_arrays)

            mapq_arrays_bool = np.sum(mapq_arrays, axis=0) / len(mapqfiles)
            
            return(mapq_arrays_bool)

        mq = return_summed_arrays()

        # open a file, where you ant to store the data
        with open('data/mapq_summary_stats.pkl', 'wb') as f:
            pkl.dump(mq, f)

In [ ]:
def get_ppfrac():
    if __name__ == "__main__":


        def return_ind_filter(array, mode):
            mapqlow = array < 30
            return mapqlow

        #for all inds
        #make arrays containing all of the coverage stats
        #then sum acdross them to get proportion of individuals pass/fail each condition

        def return_summed_arrays():

            pp_arrays = []

            for file in mapqfiles:

                pp = process_file(file, col_index = 3)

                all = process_file(file, col_index = 2)

                frac_pp = pp / all

                ppbool = frac_pp < 0.9

                pp_arrays.append(frac_pp)

            pp_arrays = np.stack(pp_arrays)

            mapq_arrays_bool = np.sum(mapq_arrays, axis=0) / len(mapqfiles)
            
            return(mapq_arrays_bool)

        mq = return_summed_arrays()

        # open a file, where you ant to store the data
        with open('data/mapq_summary_stats.pkl', 'wb') as f:
            pkl.dump(mq, f)

In [ ]:

    array_2 = da.from_zarr(reads_pp[stripped]).compute()

    prop_pp = np.nan_to_num(array_2 / array_1)

    pplow = prop_pp > 0.9

    arrays_pp_sum.append(arrays_pp_sum)

    print(f"finished calculating pp for {stripped}")

pp_arrays = np.stack(arrays_pp_sum)

pp_arrays_bool = np.sum(pp_arrays, axis=0) / 105


# open a file, where you ant to store the data
with open('data/pp_sumstats.pkl', 'w